In [ ]:
import duckdb
import pandas as pd
import numpy as np
import geopandas as gpd

# analysis and data transformaiton
import pysal.lib as ps # calculate moran
from pysal.explore import esda
import statsmodels.api as sm # for time series

# viz
from keplergl import KeplerGl
import plotly.express as px
from IPython.display import IFrame # used to make kepler works
import matplotlib.pyplot as plt # basic plot
from splot.esda import moran_scatterplot # visualize moran

In [ ]:
duckdb.sql('INSTALL spatial')
duckdb.sql('LOAD spatial')
duckdb.sql('INSTALL h3')
duckdb.sql('LOAD h3')

## Import Crimes 👮 and Community Areas 🍱 into de Jupyter Kernel

In [ ]:
# crimes & community areas
query = """
-- create com_areas
CREATE OR REPLACE TABLE com_areas AS
    SELECT *
    FROM ST_Read('../lab03/data/Com_Areas.geojson');
-- update geometry column name
ALTER TABLE com_areas 
RENAME COLUMN geom to geometry;

-- create crimes from parquet
CREATE OR REPLACE TABLE crimes AS
    SELECT *, ST_Point2D(Longitude, Latitude) as geometry 
    FROM Read_Parquet('../lab03/data/Crimes.parquet')
    WHERE Location IS NOT NULL;
"""

duckdb.sql(query)

In [ ]:
# create a gdf of community areas
query = """
SELECT 
    ST_AsHexWKB(geometry) AS geometry
FROM com_areas;
"""

df_com_geom = duckdb.sql(query).df()
gdf_com_geom = gpd.GeoDataFrame(
    df_com_geom, geometry=gpd.GeoSeries.from_wkb(df_com_geom['geometry']), crs="EPSG:4326"
)

## 📈 Mean Center trend per year

In [ ]:
query = """
SELECT 
    cr."Primary Type" as type
    , cr.Year as year
    , AVG(Latitude) as lat
    , AVG(Longitude) as lon
FROM crimes cr
JOIN com_areas ca ON ST_WITHIN(cr.geometry::GEOMETRY, ca.geometry)
GROUP BY cr."Primary Type", cr.Year
ORDER BY cr."Primary Type", cr.Year;
"""

df_mean = duckdb.sql(query).df()

In [ ]:
# visualize all types
px.scatter_map(df_mean, lat="lat", lon="lon", hover_name="year",
     color='type', zoom=13, height=800,  map_style="light")

In [ ]:
# visualize crimes with strong spatial evolution over the year 
df_theft = df_mean.query('type == "NARCOTICS"')

px.scatter_map(df_theft, lat="lat", lon="lon", hover_name="year",
     color='year', zoom=13, height=800,  map_style="light")

In [ ]:
# Visualize mean center of crimes with arrests
query = """
SELECT 
    cr.Year as year
    , AVG(Latitude) as lat
    , AVG(Longitude) as lon
FROM crimes cr
WHERE cr.Arrest IS TRUE
GROUP BY cr.Year
ORDER BY cr.Year;
"""

df_mean = duckdb.sql(query).df()

# visualize all types
px.scatter_map(df_mean, lat="lat", lon="lon", hover_name="year",
               color="year", zoom=13, height=800,  map_style="light")

## 🥅 Create and visualize a h3 index 

In [ ]:
# define a level to the h3 grid
h3_level = 8

In [ ]:
# add column h3 to crimes
query = f"""
ALTER TABLE crimes 
ADD COLUMN IF NOT EXISTS h3 VARCHAR;

UPDATE crimes
-- calculate h3 string id from lat lon level
SET h3 = h3_latlng_to_cell_string(Latitude, Longitude, {h3_level});

-- show the first 5 outputs
SELECT * FROM crimes LIMIT 5;
"""
duckdb.sql(query)

In [ ]:
# Create an empty h3 grid from community area geometries 
query = f"""
CREATE OR REPLACE TABLE h3_grid AS (
WITH hex_strs AS (
    -- for each com_areas geometries, get h3 string identifiers
    SELECT
        -- need to separate the multipolygon in simple polygons
        UNNEST(ST_Dump(geometry)).geom.h3_polygon_wkt_to_cells_string({h3_level}) as strs
    FROM  
        com_areas
)
    SELECT 
        -- create a long unique tuple with all h3 str
        UNNEST(strs) h3_str
        , UNNEST(strs).h3_cell_to_boundary_wkt() as h3_geom 
    FROM 
        hex_strs
);

-- count nb of cell
SELECT COUNT(*) as nb_cells from h3_grid;
""" 
duckdb.sql(query)

In [ ]:
# join crimes to h3 grid and create a gdf
query = """
SELECT 
    g.h3_str as hex_id,
    g.h3_geom,
    COUNT(cr.ID) as nb_crimes
FROM h3_grid g
LEFT JOIN crimes cr ON g.h3_str = cr.h3 
    AND cr.Arrest IS True
GROUP BY 
    hex_id,
    g.h3_geom;
""" 
df_h3 = duckdb.sql(query).df()

# Convert pandas DataFrame to GeoDataFrame
gdf_h3 = gpd.GeoDataFrame(
    df_h3, geometry=gpd.GeoSeries.from_wkt(df_h3['h3_geom']), crs="EPSG:4326"
)

In [ ]:
# visualize the results with a kepler maps
kepler_map = KeplerGl()
# center the map on the data
kepler_map.config['mapState'] = {
            'latitude': 41.8781,
            'longitude':  -87.6298,
            'zoom': 10
}

# add data to the map
kepler_map.add_data(data=df_h3[['hex_id', 'nb_crimes']], name='crimes')

# save to html
kepler_map.save_to_html(file_name='explore_map.html', config=kepler_map.config)

# Visualize the map in a div
IFrame(src='./explore_map.html', width=1200, height=800)

## 🍱 Spatial Autocorrelation: Moran's I

In [ ]:
# Is my distribution random or not?  

# Create Rook contiguity weights as a sparse matrix
w = ps.weights.Rook.from_dataframe(gdf_h3, use_index=True)

# Row-standardize the weights
# see https://pysal.org/libpysal/generated/libpysal.weights.W.html#libpysal.weights.W.set_transform
w.transform = 'R'

# Calculate Moran's I - Row-standardize the weights
# see https://pysal.org/esda/generated/esda.Moran.html
mi = esda.Moran(gdf_h3['nb_crimes'].astype('float64'), w)

print("------")
print(f"Moran's I: {round(mi.I, 3)}")
print(f"p-value: {round(mi.p_sim, 4)}")

In [ ]:
# you can use the splot library to visualize moran's I
moran_scatterplot(mi);

## 🌭 Hot-spot Analysis: Getis-Ord Gi* 

In [ ]:
# Create Rook contiguity weights as a sparse matrix
w = ps.weights.Rook.from_dataframe(gdf_h3, use_index=True)

# Row-standardize the weights
# see https://pysal.org/libpysal/generated/libpysal.weights.W.html#libpysal.weights.W.set_transform
w.transform = 'R'

# Compute G_Local
# see https://pysal.org/esda/generated/esda.G_Local.html
g_local = esda.G_Local(gdf_h3['nb_crimes'].astype('float64'), w)

# Extract z-scores and p-values
gdf_h3['Gi_star_zscore'] = g_local.z_sim
gdf_h3['Gi_star_pvalue'] = g_local.p_sim

In [ ]:
px.choropleth_map(gdf_h3, geojson=gdf_h3.geometry, locations=gdf_h3.index,
                          color='Gi_star_zscore', color_continuous_scale="RdBu_r",
                          hover_data=['Gi_star_zscore','Gi_star_pvalue'],
                          zoom=10, center={"lat": 41.8781, "lon": -87.6298}, 
                          height= 1000, map_style="dark"
                 )

In [ ]:
# considering the p-value

gdf_h3['cluster'] = 'Not_Significant'
# significant hot spot
gdf_h3.loc[(gdf_h3['Gi_star_pvalue'] < 0.05) & (gdf_h3['Gi_star_zscore'] > 2.0), 'cluster'] = 'Hotspot_95'
gdf_h3.loc[(gdf_h3['Gi_star_pvalue'] < 0.01) & (gdf_h3['Gi_star_zscore'] > 2.0), 'cluster'] = 'Hotspot_99'

# significant cold spot
gdf_h3.loc[(gdf_h3['Gi_star_pvalue'] < 0.05) & (gdf_h3['Gi_star_zscore'] < -1.5), 'cluster'] = 'Coldspot_95'
gdf_h3.loc[(gdf_h3['Gi_star_pvalue'] < 0.01) & (gdf_h3['Gi_star_zscore'] < -1.5), 'cluster'] = 'Coldspot_99'

# Convert 'cluster' to categorical
gdf_h3['cluster'] = gdf_h3['cluster'].astype('category')

# Define discrete color map
color_map = {
    'Not_Significant': 'lightgray',
    'Hotspot_95': 'lightcoral',
    'Hotspot_99': 'red',
    'Coldspot_95': 'blue',
    'Coldspot_99': 'darkblue'
}

px.choropleth_map(gdf_h3, geojson=gdf_h3.geometry, locations=gdf_h3.index,
                          color='cluster', color_discrete_map=color_map,
                          hover_data=['Gi_star_zscore','Gi_star_pvalue', 'cluster'],
                          zoom=10, center={"lat": 41.8781, "lon": -87.6298}, 
                          height= 1000, map_style="light"
                 )

## 🌶️ Hot-Spot Analysis: Local Moran's I

In [ ]:
# Create Rook contiguity weights as a sparse matrix
w = ps.weights.Rook.from_dataframe(gdf_h3, use_index=True)

# Row-standardize the weights
w.transform = 'R'

# Compute Moran_Local
# see https://pysal.org/esda/generated/esda.Moran_Local.html#esda.Moran_Local.__init__
moran = esda.Moran_Local(gdf_h3['nb_crimes'].astype('float64'), w, geoda_quads=True)

# Moran's I value and its significance
gdf_h3['moran_cat'] = moran.q
gdf_h3['moran_zscore'] = moran.z_sim
gdf_h3['moran_pvalue'] = moran.p_sim

In [ ]:
# you can visualize local moran values in a scattewr plot with the splot library
moran_scatterplot(moran, p=0.05);

In [ ]:
# Visualizing the Z-scores

gdf_h3['cluster_moran'] = 'Not_Significant'

gdf_h3.loc[(gdf_h3['moran_pvalue'] < 0.10) & (gdf_h3['moran_cat'] == 1), 'cluster_moran'] = 'High-High_90'
gdf_h3.loc[(gdf_h3['moran_pvalue'] < 0.10) & (gdf_h3['moran_cat'] == 2), 'cluster_moran'] = 'Low-Low_90'
gdf_h3.loc[(gdf_h3['moran_pvalue'] < 0.10) & (gdf_h3['moran_cat'] == 3), 'cluster_moran'] = 'Low-High_90'
gdf_h3.loc[(gdf_h3['moran_pvalue'] < 0.10) & (gdf_h3['moran_cat'] == 4), 'cluster_moran'] = 'High-low_90'

# Define discrete color map
color_map = {
    'Not_Significant': 'lightgray',
    'High-High_90': 'lightcoral',
    'Low-Low_90': 'lightblue',
    'Low-High_90': 'blue',
    'High-low_90': 'red',
}

px.choropleth_map(gdf_h3, geojson=gdf_h3.geometry, locations=gdf_h3.index,
                          color='cluster_moran', color_discrete_map=color_map,
                          hover_data=['moran_zscore', 'moran_pvalue', 'cluster_moran'],
                          zoom=10, center={"lat": 41.8781, "lon": -87.6298}, 
                          height= 1000, map_style="light", title="Anselin Local Moran'I of crime with arrests"
                 )

In [ ]:
# Visualizing the p-value

gdf_h3['cluster_moran'] = 'Not_Significant'

gdf_h3.loc[(gdf_h3['moran_pvalue'] < 0.10), 'cluster_moran'] = 'p-value<0.1'
gdf_h3.loc[(gdf_h3['moran_pvalue'] < 0.05), 'cluster_moran'] = 'p-value<0.95'
gdf_h3.loc[(gdf_h3['moran_pvalue'] < 0.01), 'cluster_moran'] = 'p-value<0.01'


# Define discrete color map
color_map = {
    'Not_Significant': 'lightgray',
    'p-value<0.1': 'lightgreen',
    'p-value<0.95': 'green',
    'p-value<0.01': 'darkgreen',
}

px.choropleth_map(gdf_h3, geojson=gdf_h3.geometry, locations=gdf_h3.index,
                          color='cluster_moran', color_discrete_map=color_map,
                          hover_data=['moran_zscore', 'moran_pvalue', 'cluster_moran'],
                          zoom=10, center={"lat": 41.8781, "lon": -87.6298}, 
                          height= 1000, map_style="light", title="Anselin Local Moran'I p-values for crimes with arrest"
                 )

## ⏲️ Time Series

#### 👀 Explore the time series in 3D

In [ ]:
# Visualize the time series in 3D
query = """
SELECT 
    g.h3_str as hex_id,
    h3_cell_to_latlng(g.h3_str) as geom,
    COUNT(cr.ID) as nb_crimes, 
    (cr.Year - 2001) * 100 as year 
FROM h3_grid g
LEFT JOIN crimes cr ON g.h3_str = cr.h3 
    AND cr.Arrest IS TRUE
GROUP BY 
    hex_id,
    geom,
    cr.Year;
""" 
df_h3 = duckdb.sql(query).df()

# Convert pandas DataFrame to GeoDataFrame
df_h3[['lat', 'lon']] = pd.DataFrame(df_h3['geom'].tolist(), index=df_h3.index)

In [ ]:
# create map 
kepler_map = KeplerGl()
# center the map on the data
kepler_map.config['mapState'] = {
            'latitude': 41.8781,
            'longitude':  -87.6298,
            'zoom': 10
}

# add data to the map
kepler_map.add_data(data=df_h3[['lat', 'lon', 'year', 'nb_crimes']], name='crimes')

# save to html
kepler_map.save_to_html(file_name='explore_map.html', config=kepler_map.config)

# Visualize the map in a div
IFrame(src='./explore_map.html', width=1200, height=800)

In [ ]:
px.scatter_3d(df_h3, x="lon", y="lat", z='year', size='nb_crimes', color='nb_crimes', height= 1000)

#### 📈 Visualize the time series

In [ ]:
# Visualize the time series in 3D
query = """
SELECT 
    g.h3_str as hex_id,
    h3_cell_to_latlng(g.h3_str) as geom,
    COUNT(cr.ID) as nb_crimes, 
    cr.Date as date
FROM h3_grid g
LEFT JOIN crimes cr ON g.h3_str = cr.h3 
    AND cr.Arrest IS TRUE
    AND cr.Year BETWEEN 2019 AND 2024
GROUP BY 
    hex_id,
    geom,
    date,
    cr.Year
ORDER BY date;
""" 

crimes_time_series = duckdb.sql(query).df().dropna(subset=['date'])

crimes_time_series[['lat', 'lon']] = pd.DataFrame(crimes_time_series['geom'].tolist(), index=crimes_time_series.index)

In [ ]:
# Reindex the Time Series per year
indexed_ts = crimes_time_series.copy()
indexed_ts.set_index('date', inplace=True)

In [ ]:
# visualize the time series in time

# Decompose the time series
decomposition = sm.tsa.seasonal_decompose(indexed_ts['nb_crimes'], model='additive', period=4)
fig = decomposition.plot()
fig.set_size_inches((16, 9))
# Tight layout to realign things
fig.tight_layout()
plt.show()

In [ ]:
# visualize per day count

# Resample data to get daily crime counts
daily_ts = indexed_ts.resample('D').size()

# Reset index for modeling
daily_ts = daily_ts.reset_index()
daily_ts.columns = ['date', 'nb_crimes']

px.line(daily_ts, x='date', y='nb_crimes', title='Daily Crime Counts (2019-2024)')